In [9]:
pip install httpx


  Using cached httpx-0.28.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached httpcore-1.0.9-py3-none-any.whl.metadata (21 kB)
Using cached httpx-0.28.1-py3-none-any.whl (73 kB)
Using cached httpcore-1.0.9-py3-none-any.whl (78 kB)

   ---------------------------------------- 0/2 [httpcore]
   -------------------- ------------------- 1/2 [httpx]
   ---------------------------------------- 2/2 [httpx]

Note: you may need to restart the kernel to use updated packages.


In [1]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()


print("Loaded password:", os.environ.get("OPEN_NOTEBOOK_PASSWORD"))




Loaded password: supersecret123


In [13]:
from fastapi import Depends, HTTPException

import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

from fastapi import HTTPException, Request, responses
from fastapi.security import HTTPBearer
from api.routers.auth import PasswordAuthMiddleware, check_api_passwword 
security = HTTPBearer()

@app.get("/secure")
def secure_route(request: Request, credentials: HTTPAuthorizationCredentials = Depends(security)):
    print("Headers:", request.headers)
    token = credentials.credentials
    if token != OPEN_NOTEBOOK_PASSWORD:
        raise HTTPException(status_code=401, detail="Invalid authorization")
    return {"message": "Access granted"}


In [14]:
from fastapi import FastAPI, Depends
from fastapi.security import HTTPAuthorizationCredentials
from api.routers.auth import PasswordAuthMiddleware, check_api_passwword  # your fixed auth.py

app = FastAPI()
app.add_middleware(PasswordAuthMiddleware)

@app.get("/health")
async def health():
    return {"status": "ok"}

@app.get("/secure")
async def secure(credentials: HTTPAuthorizationCredentials = Depends(check_api_passwword)):
    return {"message": "You passed the password check!"}


In [15]:
from fastapi.testclient import TestClient

client = TestClient(app)

print("Health:", client.get("/health").status_code, client.get("/health").json())

print("Secure (no auth):", client.get("/secure").status_code, client.get("/secure").json())

print("Secure (wrong pass):", client.get("/secure", headers={"Authorization": "Bearer wrongpass"}).status_code,
      client.get("/secure", headers={"Authorization": "Bearer wrongpass"}).json())

print("Secure (correct pass):", client.get("/secure", headers={"Authorization": "Bearer supersecret123"}).status_code,
      client.get("/secure", headers={"Authorization": "Bearer supersecret123"}).json())


Health: 200 {'status': 'ok'}
Secure (no auth): 401 {'detail': 'Missing authorization header'}
Secure (wrong pass): 401 {'detail': 'Missing authorization'}
Secure (correct pass): 401 {'detail': 'Missing authorization'}


In [14]:
import os
from dotenv import load_dotenv
from fastapi import FastAPI, Depends
from fastapi.security import HTTPAuthorizationCredentials
from fastapi.testclient import TestClient

# Load env
load_dotenv()
os.environ["OPEN_NOTEBOOK_PASSWORD"] = os.getenv("OPEN_NOTEBOOK_PASSWORD", "supersecret123")

# Import your auth code
from api.routers.auth import PasswordAuthMiddleware, check_api_passwword  

# Create FastAPI app
app = FastAPI()
app.add_middleware(PasswordAuthMiddleware)

@app.get("/health")
async def health():
    return {"status": "ok"}

@app.get("/secure")
async def secure(credentials: HTTPAuthorizationCredentials = Depends(check_api_passwword)):
    return {"message": "You passed the password check!"}


# Test using TestClient
client = TestClient(app)

print("Health:", client.get("/health").status_code, client.get("/health").json())

print("Secure (no auth):", client.get("/secure").status_code, client.get("/secure").json())

print("Secure (wrong pass):", 
      client.get("/secure", headers={"Authorization": "Bearer wrongpass"}).status_code,
      client.get("/secure", headers={"Authorization": "Bearer wrongpass"}).json())

print("Secure (correct pass):", 
      client.get("/secure", headers={"Authorization": f"Bearer {os.environ['OPEN_NOTEBOOK_PASSWORD']}"}).status_code,
      client.get("/secure", headers={"Authorization": f"Bearer {os.environ['OPEN_NOTEBOOK_PASSWORD']}"}).json())


Health: 200 {'status': 'ok'}
Secure (no auth): 401 {'detail': 'Missing authorization header'}
Secure (wrong pass): 401 {'detail': 'Missing authorization'}
Secure (correct pass): 401 {'detail': 'Missing authorization'}
